In [1]:
import os, sys,math,folium
import numpy as np
import pandas as pd
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
from datetime import datetime
plt.style.use('seaborn')
from folium.plugins import HeatMap
from geopy import geocoders
from geopy.geocoders import GoogleV3
from folium.plugins import MarkerCluster
import bs4,re,requests

In [168]:
syrup=pd.read_csv("bce/syrupSheet1.csv",encoding="ISO-8859-1")

#Problems with porting dates past 8/08/17
#syrup=syrup.iloc[:1205,:]

syrup.Brand=syrup.Brand.apply(lambda x: str(x).lower())

syrup['Promo']=syrup['Promo'].apply(lambda x: str(x).lower())

#Clean up promo columns
syrup=syrup[syrup.Promo.str.contains('remove')==False]
syrup=syrup[syrup.Promo.str.contains('ended')==False]
syrup=syrup[syrup.Promo.apply(lambda x: len(x)>15)]

exProvs=['quebec','manitoba',"saskatchewan",'mb:','qc','sk:']

for i in exProvs:
    syrup=syrup[syrup.Promo.str.contains(i)==False]

#Exclude brands that are not very relevant
ex=["7-eleven speakout",'eastlink','cityfone','bell mts','petro-canada',
   'saskTel','7-eleven','petro canada','bellmts','sasktel','vidéotron',
   'videotron','nan','pc mobile']

for i in ex:
    syrup=syrup[syrup.Brand!=i]

master={"Brand":[],"Promo":[],"Start":[],"End":[]}


for brand in syrup.Brand.unique():
    brandDf=syrup[syrup.Brand==brand]
    promos=brandDf.Promo.unique()
    
    for i in promos:

        temp=brandDf[brandDf.Promo==i]
        endDate=temp.dIndex.max()
        startDate=temp.dIndex.min()
        
        master['Brand'].append(brand)
        master['Promo'].append(i)
        master['Start'].append(startDate)
        master['End'].append(endDate)
    


df=pd.DataFrame(master)
df.dropna(inplace=True)
df['Promo']=df.Promo.str.capitalize()

#Get dummies
temp=pd.get_dummies(df.Start)+pd.get_dummies(df.End)

for h in range(len(temp)):
    start=False
    end=False
    for i,c in enumerate(temp.columns):
        if temp.loc[h,c]==1 and not start:
    
            start=True
        elif temp.loc[h,c]==1 and start:
            end=True
            break
        if start and not end:
            temp.loc[h,c]=1

temp.dropna(axis=1)

colors={'bell':'b','chatr':'r','fido':'r','freedom mobile':'o','koodo mobile':'g',
       'public mobile':'g','rogers':'r','telus':'g','virgin mobile':'r'}

df['Colors']=df.Brand.apply(lambda x: colors[x])

            
df=pd.concat([df,temp],axis=1)

indexMap=pd.DataFrame({'Date':syrup.Date.unique(),'dIndex':syrup.dIndex.unique()})
indexMap.set_index('dIndex',inplace=True)

temp=dict(indexMap['Date'])

df.rename(columns=temp,inplace=True)
df.fillna(0,inplace=True)



df.to_clipboard()


In [103]:
df.Brand.unique()

array(['bell', 'chatr', 'fido', 'freedom mobile', 'koodo mobile',
       'pc mobile', 'public mobile', 'rogers', 'telus', 'virgin mobile'], dtype=object)

In [165]:
df

,Brand,End,Promo,Start,Colors,4/12/17,4/19/17,4/25/17,5/01/17,5/08/17,...,12/18/17,1/03/18,1/09/18,1/16/18,1/22/18,1/29/18,2/05/18,2/12/18,2/20/18,2/27/18
0,bell,45,$150 trade-in bonus on the samsung galaxy s9 o...,45,b,0,0,0,0,0,...,0,0,0,0,0,0.0,0,0,0,2
1,bell,45,Free samsung galaxy tab e with samsung galaxy ...,44,b,0,0,0,0,0,...,0,0,0,0,0,0.0,0,0,1,1
2,bell,45,2gb bonus on 10gb share plan (main regions),40,b,0,0,0,0,0,...,0,0,0,0,1,1.0,1,1,1,1
3,bell,45,$100 trade-in credit with selected phone,33,b,0,0,0,0,0,...,1,1,1,1,1,1.0,1,1,1,1
4,bell,45,Save $10/month with every additional line on a...,40,b,0,0,0,0,0,...,0,0,0,0,1,1.0,1,1,1,1
5,bell,44,Increase price (+$5) with every plan options.,44,b,0,0,0,0,0,...,0,0,0,0,0,0.0,0,0,2,0
6,bell,39,Save $10/month with every additional line on a...,39,b,0,0,0,0,0,...,0,0,0,2,0,0.0,0,0,0,0
7,bell,39,2gb bonus on all data options on main regions,38,b,0,0,0,0,0,...,0,0,1,1,0,0.0,0,0,0,0
8,bell,38,Get $10/mth off for 12 mths when you bring you...,38,b,0,0,0,0,0,...,0,0,2,0,0,0.0,0,0,0,0
9,bell,37,Up to 3gb bonus on selected data plans,36,b,0,0,0,0,0,...,1,1,0,0,0,0.0,0,0,0,0
